# Test Driven Model Development Using a Nice Sphere
Tim Tyree<br>
6.26.2020

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
sys.path.append("../lib") 
from lib import *
sys.path.append("lib") 
from lib import *

# from operari import *
# from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: consider speeding up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse

#formating
from matplotlib import rc
rc('text', usetex=True)

%autocall 1
%load_ext autoreload
%autoreload 2

# dev of simulation

## DONE: import a mesh of the unit 2-sphere

In [ ]:
os.chdir(f'{nb_dir}/Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_64.stl')
#subtract the center of mass
mesh.vertices -= mesh.center_mass
#normalize the mean radius to 1
V_initial = 26 #cm^3
# mesh.vertices /= np.cbrt((mesh.volume)*3/(4*np.pi))
mesh.vertices *= np.cbrt(V_initial/mesh.volume)
#normalize the mean radius to R
# R = 1. #unit length
# mesh.vertices *= R/np.mean(np.linalg.norm(mesh.vertices,axis=1))

In [ ]:
assert(mesh.is_watertight)
assert(mesh.is_winding_consistent)

In [ ]:
# mean radius check
print (f"initial volume is {mesh.volume:.1f} cm^3.")
print (f"which has a mean radius of {np.mean(np.linalg.norm(mesh.vertices,axis=1)):.1f} cm.")


### [deprecated, just use real space, dude.]

In [ ]:
#doesn't work. don't know why
# def mat_to_str(A):
#     string = f'''
#     {A[[0,0]]} & {A[[0,1]]} & {A[[0,2]]} & {A[[0,3]]} \\ 
#     {A[[1,0]]} & {A[[1,1]]} & {A[[1,2]]} & {A[[1,3]]} \\ 
#     {A[[2,0]]} & {A[[2,1]]} & {A[[2,2]]} & {A[[2,3]]} \\ 
#     {A[[3,0]]} & {A[[3,1]]} & {A[[3,2]]} & {A[[3,3]]} \\ 
#     '''
#     latex_string = r'$\left( \begin{array}{ll} ' + string + r'\end{{array}} \right)$'
#     return latex_string
# # text(my_matrix, (1,1))

### [deprecated, use real space] cast all vertices and shapes to quaternions in numpy. record as the undeformed surface
- TODO: dilate the sphere by a known amount
- TODO: compute the deformation gradient
- TODO: compute the 1st piola-kirchoff tensor for some constitutive relation
- TODO: taking thickness to be $\delta\ll1$, compute the effective surface tension, $\gamma$

In [ ]:
from colorsys import hls_to_rgb

def colorize(z):
    n,m = z.shape
    c = np.zeros((n,m,3))
    c[np.isinf(z)] = (1.0, 1.0, 1.0)
    c[np.isnan(z)] = (0.5, 0.5, 0.5)

    idx = ~(np.isinf(z) + np.isnan(z))
    A = (np.angle(z[idx]) + np.pi) / (2*np.pi)
    A = (A + 0.5) % 1.0
    B = 1.0 - 1.0/(1.0+abs(z[idx])**0.3)
    c[idx] = [hls_to_rgb(a, b, 0.8) for a,b in zip(A,B)]
    return c

In [ ]:
#define the constant pauli matrices as a basis for real space
sigx = 1.j*np.array([[0.,1.],[1.,0.]],dtype=complex)
sigy = 1.j*np.array([[0.,-1.j],[1.j,0.]],dtype=complex)
sigz = 1.j*np.array([[1.,0.],[0.,-1.]],dtype=complex)
one  = np.array([[1.,0.],[0.,1.]],dtype=complex)
zero = np.array([[0.,0.],[0.,0.]],dtype=complex)

#print the imaginary spatial basis, and test that identity and zero multiply correctly.
print(r'$\sigma_x is $')
print(sigx)
print(r'$\sigma_y is $')
print(sigy)
print(r'$\sigma_z is $')
print(sigz)
# print(one)
# print(zero)
# print((one.dot(sigx)))
assert((one.dot(sigx)==sigx).all())
assert((zero.dot(sigx)==zero).all())

# @njit
def vec_to_quaternion(v):
    #define the constant pauli matrices as a basis for real space
    #     sigx = np.array([[0.,1.],[1.,0.]],dtype=complex)
    #     sigy = np.array([[1.j,0.],[0.,-1.j]],dtype=complex)
    #     sigz = np.array([[1.,0.],[0.,-1.]],dtype=complex)
    return v[0]*sigx + v[1]*sigy + v[2]*sigz

In [ ]:
#test the inverse of the basis elements are minus the basis elements
assert((np.linalg.inv(sigx)==-sigx).all())
assert((np.linalg.inv(sigy)==-sigy).all())
assert((np.linalg.inv(sigz)==-sigz).all())
#test the basis elements are anticommuting
assert((sigx*sigy-sigy*sigx==zero).all())
assert((sigx*sigz-sigz*sigx==zero).all())
assert((sigz*sigy-sigy*sigz==zero).all())


In [ ]:
trim = mesh.triangles[0]
tris = mesh.triangles[100]

dm1 = trim[1]-trim[0]
dm2 = trim[2]-trim[0]
Am =  np.cross(dm1,dm2)/2

ds1 = tris[1]-tris[0]
ds2 = tris[2]-tris[0]
As =  np.cross(ds1,ds2)/2


qm1 = dm1[0]*sigx + dm1[1]*sigy + dm1[2]*sigz
qm2 = dm2[0]*sigx + dm2[1]*sigy + dm2[2]*sigz
qs1 = ds1[0]*sigx + ds1[1]*sigy + ds1[2]*sigz
qs2 = ds2[0]*sigx + ds2[1]*sigy + ds2[2]*sigz

qAm =  Am[0]*sigx + Am[1]*sigy + Am[2]*sigz
qAs =  As[0]*sigx + As[1]*sigy + As[2]*sigz

# solve for the deformation gradient between either pair of edges.
F1 = qs1.dot(np.linalg.inv(qm1))
F2 = qs2.dot(np.linalg.inv(qm2))
# FA = qAs.dot(np.linalg.inv(qAm))

In [ ]:
#singular value decomposition on the block diagonal deformations
U1, S1, V1 = np.linalg.svd(F1)
V1 = V1.T.conjugate()

U2, S2, V2 = np.linalg.svd(F2)
V2 = V2.T.conjugate()

In [ ]:
#verify svd is returning hermitian factors
assert((np.around(U1.dot(U1.T.conjugate()),3)==one).all())
assert((np.around(V1.dot(V1.T.conjugate()),3)==one).all())

In [ ]:
Dm = np.array(((qm1,zero),(zero,qm2))).reshape((4,4))
Ds = np.array(((qs1,zero),(zero,qs2))).reshape((4,4))

In [ ]:
#TODO: test if rotating by F maps dm to ds
F1.dot(qm1).dot(F1.conjugate().T)

In [ ]:
np.linalg.svd(Dm)

In [ ]:
#TODO: visualize F1 and F2 and see if they make sense.

TODO: matrix to quaternion

TODO: quaternion to matrix

In [ ]:
sigy.T.conjugate()

In [ ]:
sigy

### [deprecated, use real space] Handling Stress with quaternions only with nearly conformal mappings

Two deformations are conformal if their differentials are related by a rotation and a scaling.

$$d\tilde f = e^uRdf$$

Rotations represented in $\mathbb{R}^3$ require quadratic constraints which produce shear stress, which causes angles change under deformation.  The action of a quaternion on a vector can be written in terms of a rotation and a scale.

The shape of a triangle, which has six degrees of freedom in $\mathbb{R}^3$ cannot be represented by a single quaternion, which has only four degrees of freedom.  It is therefore necessary to represent the shape of a triangle with at least two quaternions, and it is sufficient to constrain those quaternions to be imaginary, since Im$\,\mathbb{H}\simeq\mathbb{R}^3$.

## DONE: test based development of geom_func.py

## DONE: test based development of rotating two triangles to be coplanar

In [ ]:
# tris = mesh.triangles[110]
tris = mesh.triangles[243]
trim = mesh.triangles[321]

In [ ]:
# #DONE: copy the 'rotate into same plane solution' to a .py file
# #DONE: rotate two triangles into the same plane with get_R
# testing=True
# def get_R(trim,tris,testing=True):
#     '''trim is a triangle in material/reference space that is 
#     deformed to tris, which is a triangle in real space.  
#     returns the 3x3 rotation matrix aligning both their area normals and their first shape vector.
#     get_R assumes the deformation is continuous and did not invert the triangle being deformed.'''
#     dm1 = trim[1]-trim[0]     #precompute in final algorithm
#     dm2 = trim[2]-trim[0]     #precompute in final algorithm
#     Am =  np.cross(dm1,dm2)/2 #precompute in final algorithm

#     ds1 = tris[1]-tris[0]
#     ds2 = tris[2]-tris[0]
#     As =  np.cross(ds1,ds2)/2

#     #Ra is a rotation_matrix that rotation_matrix rotates Ashat onto Amhat
#     Ashat = As/np.linalg.norm(As)
#     Amhat = Am/np.linalg.norm(Am)
#     v2 = Ashat
#     v1 = Amhat
#     axisa  = np.cross(v1,v2)
#     thetaa = np.arcsin(np.linalg.norm(axisa))
#     Ra = rotation_matrix(axisa, thetaa)
#     if not np.isclose(np.abs(np.dot(np.dot(Ra, Amhat),Ashat)),1.).all(): #doesn't care if area's end up flipped
#         Ra = Ra.T
#     if testing:
#         assert(np.isclose(np.abs(np.dot(np.dot(Ra, Amhat),Ashat)),1.).all())

#     #Rb is a rotation_matrix that rotates the Ra*dm1 onto ds1 without unaligning the area vectors
#     v1 = np.dot(Ra,dm1/np.linalg.norm(dm1))
#     v2 = ds1/np.linalg.norm(ds1)
#     axisb  = Ashat
#     thetab = np.arccos(np.dot(v1,v2))
#     Rb = rotation_matrix(axisb, thetab).T


#     if not np.isclose(np.dot(Rb, v1),v2).all():
#         Rb = Rb.T

#     if testing:
#         # test that Rb keeps the area vectors aligned
#         assert(np.isclose(np.dot(Rb, v1),v2).all())
#         assert(np.isclose(np.abs(np.dot(np.dot(Ra, Amhat),Ashat)),1.).all())

#     R = Rb.dot(Ra)
#     if testing:
#         # test that R = Rb.dot(Ra).T rotates Amhat onto Ashat
#         assert(np.isclose(np.abs(np.dot(R.dot(Amhat),Ashat)),1.).all())
#         # test that R = (Rb*Ra).T rotates dm1 onto ds1
#         assert(np.isclose(R.dot(dm1/np.linalg.norm(dm1)),ds1/np.linalg.norm(ds1)).all())
#     return R

In [ ]:
# # np.abs(np.dot(
# print(R.dot(Amhat))
# print(Ashat)

# print(R.dot(dm1/np.linalg.norm(dm1)))
# print(ds1/np.linalg.norm(ds1))

### test based development of aligning two triangles

In [ ]:
# def align_triangles(trim,tris, testing=True):
#     '''return parameters for aligning trim to tris.  
#     coplanar positions are returned for trim before any shear deformation, 
#     but contracting the first edges to match.'''
#     dm1 = trim[1] - trim[0]  #precompute in final algorithm
#     dm2 = trim[2] - trim[0]  #precompute in final algorithm
#     Am = np.cross(dm1, dm2) / 2  #precompute in final algorithm

#     ds1 = tris[1] - tris[0]
#     ds2 = tris[2] - tris[0]
#     As = np.cross(ds1, ds2) / 2

#     #Ra is a rotation_matrix that rotation_matrix rotates Ashat onto Amhat
#     Ashat = As / np.linalg.norm(As)
#     Amhat = Am / np.linalg.norm(Am)

#     R = get_R(trim, tris, testing=testing)
#     if testing:
#         assert (np.isclose(np.linalg.norm(R.dot(dm1)) / np.linalg.norm(dm1), 1.))

#     #test that the local "x" axis is aligned by R
#     xhat = ds1 / np.linalg.norm(ds1)

#     #yhat is not needed for energy calculation, but is needed for deformation gradient calculation via outer product
#     yhat = np.cross(As, xhat)
#     yhat /= np.linalg.norm(yhat)

#     #scale so "the first" edge matches between the two triangles
#     xi1 = np.linalg.norm(ds1) / np.linalg.norm(dm1)
    
#     if testing:
#         # Test that the first vectors match.
#         assert (np.isclose(xhat, R.dot(dm1) / np.linalg.norm(dm1)).all())
#         assert (np.isclose(np.linalg.norm(xi1 * R.dot(dm1)), np.linalg.norm(ds1)))

#     # project all  edges onto the first vector and the second vector
#     xs1 = xhat.dot(ds1)  # full length
#     ys1 = yhat.dot(ds1)  # zero
#     xs2 = xhat.dot(ds2)
#     ys2 = yhat.dot(ds2)

#     # for each second vector, compute the orthogonal component
#     xm1 = xhat.dot(xi1 * R.dot(dm1))
#     ym1 = yhat.dot(xi1 * R.dot(dm1))
#     xm2 = xhat.dot(xi1 * R.dot(dm2))
#     ym2 = yhat.dot(xi1 * R.dot(dm2))

#     if testing:
#         # test that nothing's left out of plane using the pythagorean theorem
#         assert (np.isclose(np.sqrt(xm1**2 + ym1**2), np.linalg.norm(xi1 * dm1)))
#         assert (np.isclose(np.sqrt(xm2**2 + ym2**2), np.linalg.norm(xi1 * dm2)))
#         assert (np.isclose(np.sqrt(xs1**2 + ys1**2), np.linalg.norm(ds1)))
#         assert (np.isclose(np.sqrt(xs2**2 + ys2**2), np.linalg.norm(ds2)))

#     # scale the triangle heights to match
#     xi2 = ys2 / ym2

#     #use a shear deformation from the heisenburg group to make the triangles match
#     s = (xs2 - xm2) / ys2



#     return xm2, xm1, ym2, ym1, xs2, xs1, ys2, ys1, xhat, yhat, xi1, xi2, s


## DONE: test based development of deforming one triangle onto any coplanar triangle

In [ ]:
tris = mesh.triangles[137]
trim = mesh.triangles[153]
xm2, xm1, ym2, ym1, xs2, xs1, ys2, ys1, xhat, yhat, xi1, xi2, s = align_triangles(trim,tris)

In [ ]:
xm_values = [0., xm2, xm1, 0.]
ym_values = [0., ym2, ym1, 0.]
xs_values = [0., xs2, xs1, 0.]
ys_values = [0., ys2, ys1, 0.]

# with 2D projections in hand, plot some vectors on top of eachother in two different colors.
plt.plot(xs_values, ys_values, 'k')
plt.plot(xm_values, ym_values, 'b')
plt.show()

In [ ]:
#TODO: test ^this by ploting a number of triangles.  do they consistently match?
xm_values = [0., xm2 + xi2*s*ym2, xm1, 0.]
ym_values = [0., xi2*ym2, ym1, 0.]
xs_values = [0., xs2, xs1, 0.]
ys_values = [0., ys2, ys1, 0.]
plt.plot(xs_values, ys_values, 'k')
plt.plot(xm_values, ym_values, 'b')
plt.show()

In [ ]:
# # compute the 3D strain gradient
# def make_S_3by3(xi1, xi2, s, xhat, yhat):
#     '''returns the strain gradient in the global basis of the dynamical space.
#     s = ( xs2 - xm1 ) / ys2
#     xi2 = ys2 / ym2
#     xi1 = norm(ds1)/norm(dm1)
#     xm, ym have been rotated and scaled to be coplanar with ds1 and ds2.  '''
#     projx = np.outer(xhat,xhat)
#     projy = np.outer(yhat,yhat)
#     projy_to_x = np.outer(xhat,yhat)    
#     S = np.eye(3) + (xi2-1) * projy + xi2*s * projy_to_x 
#     S *= xi1
#     return S

# # compute the 2D strain gradient
# def make_S_2by2(xi1, xi2, s):
#     '''returns the strain gradient in the local basis of the dynamical space.
#     s = ( xs2 - xm2 ) / ys2
#     xi2 = ys2 / ym2
#     xi1 = norm(ds1)/norm(dm1)
#     xm, ym have been rotated and scaled to be coplanar with ds1 and ds2.  '''
#     return xi1 * np.array([[1., s*xi2],[0., xi2]])

In [ ]:
tris = mesh.triangles[137]
trim = mesh.triangles[153]

xm2, xm1, ym2, ym1, xs2, xs1, ys2, ys1, xhat, yhat, xi1, xi2, s = align_triangles(trim,tris)

In [ ]:
#TODO: test ^this by ploting a number of triangles.  do they consistently match?
xm_values = [0., xm2 + xi2*s*ym2, xm1, 0.]
ym_values = [0., xi2*ym2, ym1, 0.]
xs_values = [0., xs2, xs1, 0.]
ys_values = [0., ys2, ys1, 0.]
plt.plot(xs_values, ys_values, 'k')
plt.plot(xm_values, ym_values, 'b')
plt.show()

## DONE: testing the explicit deformation map

$$x= \phi(X)= FX + b$$
$$F = S R$$
$$b = tris[0] - trim[0]$$

In [ ]:
# # collect the operations into polar decomposition of deformation gradient matrix.
# def get_SR(trim,tris, printing=True, testing=False):
#     R = get_R(trim,tris,testing=testing)
#     retval = align_triangles(trim,tris, testing=testing)
#     #TODO: make retval more compact
#     xm2, xm1, ym2, ym1, xs2, xs1, ys2, ys1, xhat, yhat, xi1, xi2, s = retval
#     S = make_S_3by3(xi1, xi2, s, xhat, yhat)
#     if (xi2<0) and printing:
#         print('xi2 is negative.')
#     if (ym2<0) and printing:
#         print('ym2 is negative.')
#     return S, R    
      
# def get_F(trim,tris, printing=True, testing=False):
#     S, R = get_SR(trim,tris, printing=printing, testing=testing)
#     F = S.dot(R)
#     return F


In [ ]:
# # the explicit deformation map
# def phi(F,X,b):
#     return F.dot(X)  + b
# def get_phi(trim,tris):
#     F = get_F(trim,tris)
#     b = tris[0] - F.dot(trim[0])
#     return lambda X: phi(F,X,b)

In [ ]:
# test the explicit deformation map for a number of triangles
tris = mesh.triangles[71]
trim = mesh.triangles[30]
mtos = get_phi(trim,tris)
trim_mapped = np.array([mtos(trim[0]),mtos(trim[1]),mtos(trim[2])])
print('tris is')
print(tris)
print('trim is mapped to')
print(trim_mapped)
print('difference after mapping is')
print(tris - trim_mapped)
assert(np.isclose(tris - trim_mapped,0.).all())
# print(phi(F,trim[0],b))
# print(phi(F,trim[1],b))
# print(phi(F,trim[2],b))

## DONE: compute the piola-kirchoff stress tensor for the corotated linear elastic constitutive relation.  Then compute the nodal forces for a triangle stretched by a known amount.


The explicit deformation map is

$$\vec{x}= \vec{\phi}(\vec{X}; \vec{x})= \mathbf{F}(\vec{X}; \vec{x})\cdot \vec{X} + \vec{b}(\vec{X}; \vec{x})$$

where

$$\mathbf{F}(\vec{X}; \vec{x}) = \mathbf{S}\cdot\mathbf{R}$$ 
and 
$$\vec{b}(\vec{X}; \vec{x}) = tris[0] - \mathbf{F}(\vec{X}; \vec{x}) \cdot trim[0].$$

$S$ this is all that's needed to compute the elastic force for a given constitutive relation that's homogeneous and isotropic.  We suppose rotational invariance in the following elastic energy density:

$$
\Psi(F=SR) = \mu||S-\mathbb{1}||^2 + \frac{\lambda}{2}\text{tr}^2(S-\mathbb{1})
$$

Where $\mu$ and $\lambda$ are the first and second Lamé parameters, measuring resistance to stretching/shearing and volume change, respectively.

It can be shown this results in the following first Piola-Kirchoff stress tensor:

$$
\mathbf{P}(\mathbf{F})= 2\mu(\mathbf{S}-\mathbb{1})\mathbf{R} + \lambda \text{tr}(\mathbf{S}-\mathbb{1})\mathbf{R}
$$

There is the following conversion to Lamé's parameters from Young's modulus, $k$, and Poisson's ratio, $\nu$:

$$
\mu = \frac{k}{2(1+\nu)},\qquad \lambda=\frac{k\nu}{(1+\nu)(1-2\nu)}
$$


In [ ]:
# def get_calc_P(mu, lam, one, delta):
#     '''returns the first Piola-Kirchoff stress tensor ( times the constant membrane thickness, delta) 
#     from the corotated linear constitutive model for elastic stress.
#     Example Usage - given deformation matrix F = S.dot(R):
#     mu = 1.; lam = 1.; delta = 0.1; one = np.eye(3);
#     calc_P = get_calc_P(mu, lam, one, delta)
#     P = calc_P(S, R)'''
#     return lambda S, R: delta * (2*mu*(S - one).dot(R) + lam*np.trace(S - one) * R)

# def calc_outward_normals(trim):
#     '''
#     returns the outward unit normal vectors for the triangle, trim.
#     trim is a 3x3 numpy array.
#     '''
#     #compute local unit vectors of triangle's shape and center of mass (com)
#     d1, d2, A = get_shape(trim)
#     c   = trim[2] - trim[1]
#     c  /= np.linalg.norm(c)
#     d1 /= np.linalg.norm(d1)
#     d2 /= np.linalg.norm(d2)
#     A  /= np.linalg.norm(A)
#     com     = (trim[0]+trim[1]+trim[2])/3

#     #precompute the outward normals in material space
#     #and test that the outward normals are indeed outward
#     N1 = np.cross(d1,A)
#     N1tilde = (trim[0]+trim[1])/2 - com
#     if N1.dot(N1tilde)<0:
#         N1 *= -1
#         print("N1 was flipped.")
#     N2 = np.cross(-d2,A)
#     N2tilde = (trim[0]+trim[2])/2 - com
#     if N2.dot(N2tilde)<0:
#         N2 *= -1
#         print("N2 was flipped.")
#     N3 = np.cross(c,A)
#     N3tilde = (trim[2]+trim[1])/2 - com
#     if N3.dot(N3tilde)<0:
#         N3 *= -1
#         print("N3 was flipped.")
#     return N1,N2,N3

### functional computation of nodal forces due to elasticity

In [ ]:
#compute P, first Piola-Kirchoff stress tensor
mu  = 1. #first  lame parameter, kPa
lam = 1. #second lame parameter, kPa
delta = .01 #small thickness of membrane
one = np.eye(3)
calc_P = get_calc_P(mu, lam, one, delta)
def calc_elastic_force(trim, tris, calc_P, N1N2N3):
    '''N1N2N3 are the precomputed outward normals of trim returned by compute_outward_normals
    calc_P is returned by get_calc_P,
    tris and trim are 3x3 numpy arrays of floats'''
    S, R = get_SR(trim,tris)
    P    = calc_P(S, R)
    N1,N2,N3 = N1N2N3  #TODO: precompute these if they're too slow

    #compute the net force on each side of tris
    D1 = np.linalg.norm(tris[1]-tris[0])
    D2 = np.linalg.norm(tris[2]-tris[0])
    C  = np.linalg.norm(tris[2]-tris[1])
    F1 = - P.dot(N1) * D1
    F2 = - P.dot(N2) * D2
    F3 = - P.dot(N3) * C

    #compute the equivalent nodal forces on each vertex of tris
    nf0 = (F1 + F2)/2.  #the nodal force on the vertex at tris[0]
    nf1 = (F1 + F3)/2.  #the nodal force on the vertex at tris[1]
    nf2 = (F3 + F2)/2.  #the nodal force on the vertex at tris[2]
    return nf0, nf1, nf2

In [ ]:
calc_elastic_force(trim, tris, calc_P,calc_outward_normals(trim))

In [ ]:
trim = mesh.triangles[153]

#dilate trim to make a tris with a predicted form
sscale = 2.
tris = trim*sscale

# tris = mesh.triangles[71]
# d1, d2, A = get_shape(trim)
# com     = (trim[0]+trim[1]+trim[2])/3
# (trim[0]-com)
# np.array([
#     trim[0] + d1
#     trim[0] + d2,
#     ,
# ])

In [ ]:
#compute P, first Piola-Kirchoff stress tensor
mu  = 1. #first  lame parameter, kPa
lam = 1. #second lame parameter, kPa
delta = .01 #small thickness of membrane
one = np.eye(3)
S, R = get_SR(trim,tris)
P = calc_P(S, R)
N1,N2,N3 = calc_outward_normals(trim)

In [ ]:
#compute the net force on each side of tris
D1 = np.linalg.norm(tris[1]-tris[0])
D2 = np.linalg.norm(tris[2]-tris[0])
C  = np.linalg.norm(tris[2]-tris[1])
F1 = - P.dot(N1) * D1
F2 = - P.dot(N2) * D2
F3 = - P.dot(N3) * C

In [ ]:
# #compute the equivalent nodal forces on each vertex of tris
nf0, nf1, nf2 = calc_elastic_force(trim, tris, calc_P, calc_outward_normals(trim))
# nf0 = (F1 + F2)/2. 
# nf1 = (F1 + F3)/2.
# nf2 = (F3 + F2)/2.

In [ ]:
#compute the deformed triangle
ds1, ds2, As = get_shape(tris)
dm1, dm2, Am = get_shape(trim)

#test that the local "x" axis is aligned by R
xhat = ds1 / np.linalg.norm(ds1)

#yhat is not needed for energy calculation, but is needed for deformation gradient calculation via outer product
yhat = np.cross(As, xhat)
yhat /= np.linalg.norm(yhat)

xs1 = xhat.dot(ds1)  # full length
ys1 = yhat.dot(ds1)  # zero
xs2 = xhat.dot(ds2)
ys2 = yhat.dot(ds2)

xm1 = xhat.dot(R.dot(dm1))  # full length
ym1 = yhat.dot(R.dot(dm1))  # zero
xm2 = xhat.dot(R.dot(dm2))
ym2 = yhat.dot(R.dot(dm2))

xshift = (xs2+xs1)/6
yshift = (ys2+ys1)/6
xs_values = [0., xs2, xs1, 0.]
ys_values = [0., ys2, ys1, 0.]
xm_values = np.array([0., xm2, xm1, 0.]) + xshift
ym_values = np.array([0., ym2, ym1, 0.]) + yshift

In [ ]:
#plot up those nodal force examples.  Are they sensible?
scale_arrows = 3.2
plt.plot(xs_values, ys_values, 'k')
plt.plot(xm_values, ym_values, 'b')


#nodal forces
plt.arrow(0,0,nf0.dot(xhat)*scale_arrows,nf0.dot(yhat)*scale_arrows, color='red', head_width=.01)
plt.arrow(xs1,ys1,nf1.dot(xhat)*scale_arrows,nf1.dot(yhat)*scale_arrows, color='green', head_width=.01)
plt.arrow(xs2,ys2,nf2.dot(xhat)*scale_arrows,nf2.dot(yhat)*scale_arrows, color='blue', head_width=.01)

#forces on the sides of triangles
plt.arrow(0+xs1/2,0+ys1/2,F1.dot(xhat)*scale_arrows,F1.dot(yhat)*scale_arrows, color='black', head_width=.02)
plt.arrow(0+xs2/2,0+ys2/2,F2.dot(xhat)*scale_arrows,F2.dot(yhat)*scale_arrows, color='black', head_width=.02)
plt.arrow(xs1/2+xs2/2,ys1/2+ys2/2,F3.dot(xhat)*scale_arrows,F3.dot(yhat)*scale_arrows, color='black', head_width=.02)


plt.title('nodal forces due to stretching', fontsize=20)
plt.axis('off')
# plt.plot(xm_values, ym_values, 'b')
plt.show()
# plt.savefig(f"{nb_dir}/Figures/nodal_forces_example3.png",dpi=400)


In [ ]:
# test that the nodal forces are in plane with the deformed triangle
assert(np.isclose(As.dot(nf0),0.))
assert(np.isclose(As.dot(nf1),0.))
assert(np.isclose(As.dot(nf2),0.))
assert(np.isclose(As.dot(F1),0.))
assert(np.isclose(As.dot(F2),0.))
assert(np.isclose(As.dot(F3),0.))

In [ ]:
print(*calc_elastic_force(trim, tris, calc_P))

A Possible Future Glitch could occur if there's a lot fo twisting, which I don't expect
- if R.dot(Amhat).dot(Ashat)<0 and/or 
- if xi2 < 0

Plot force vectors, frequently and consider printing when ^these conditions occur.

## DONE: the net forces for each vertex
- then plot net forces on a sphere
- then add pressure forces
- then measure some representative values
- then use FEI, and plot those values as the sphere undergoes isobaric expansion.

### Compute pressure forces for each node
TODO(for better performance): compute nodal pressure force for each triangle, then just iterate over all triangles once

In [ ]:
# compute the pressure forces on each vertex
#using my Astar method
pressure = 1. #the pressure difference between the inside and the outside of the mesh
pres sure_forces = np.array([get_A(vid, mesh) for vid in range(mesh.vertices.shape[0])])
pressure_forces *= pressure
pressure_forces[0]

# # the mean normals of the faces the vertex is used in.
# P = 1. #the pressure difference between the inside and the outside of the mesh
# pressure_forces = np.array([mesh.vertex_normals[vid] for vid in range(mesh.vertices.shape[0])])
# pressure_forces *= P

In [ ]:
FP_lst = []
for F in pressure_forces:
    FP_lst.append(np.linalg.norm(F))

In [ ]:
fontsize=20
plt.hist(FP_lst, bins=30)
plt.title('distribution of net pressure \nforce magnitudes at vertices', fontsize=fontsize)
plt.ylabel('freq.', fontsize=fontsize)
plt.xlabel('Force (force units)',fontsize=fontsize)
plt.show()

### net elastic forces by iterating over the triangular faces

In [ ]:
# where each trim is tris*sscale
sscale = 0.5

In [ ]:
# this takes 0.5 seconds to compute the net forces on 500 rows.  Not ideal
net_elastic_forces = 0.*np.array(mesh.vertices)
for fid, face in enumerate(mesh.faces):
    vid0,vid1,vid2 = mesh.faces[fid]
    tris = mesh.triangles[fid]
    trim = tris.copy()*sscale
    nf0, nf1, nf2 = calc_elastic_force(trim, tris, calc_P)
    net_elastic_forces[vid0] += nf0
    net_elastic_forces[vid1] += nf1
    net_elastic_forces[vid2] += nf2

In [ ]:
F_lst = []
for F in net_elastic_forces:
    F_lst.append(np.linalg.norm(F))
F_lst = np.array(F_lst)


In [ ]:
plt.hist(F_lst, bins=30)
plt.title('distribution of net spring force magnitudes',fontsize=fontsize)#\nfor $\gamma=0.5$ and $k=1$ force/length units')
plt.ylabel('freq.',fontsize=fontsize)
plt.xlabel('Force (force units)',fontsize=fontsize)
plt.show()

### angles between net spring forces and net pressure forces

In [ ]:
angle_lst = []
for v1,v2 in zip(pressure_forces,net_elastic_forces):
    angle_lst.append(angle_between(v1,v2))

In [ ]:
plt.hist(angle_lst, bins=30)
plt.title('distribution of angles between\npressure forces and net elastic forces',fontsize=fontsize)
plt.ylabel('freq.',fontsize=fontsize)
plt.xlabel('angle (degrees)',fontsize=fontsize)
plt.show()

# dev for force array calculation

In [ ]:

# print(pressure)
#compute nodal pressure forces using my Astar method - sufficiently fast.
# pressure = .1 #the pressure difference between the inside and the outside of the mesh
# pressure_forces = pressure * np.array([get_A(vid, mesh) for vid in range(mesh.vertices.shape[0])])

In [ ]:
#assign model parameters
# del x0_constant_mat, k_constant_mat

#assign params for isobaric expansion experiment
# P = 1.   #    5.*1333.    #gcm/s^2 / cm^2
# mu = .1; lam = .1; delta = 0.1; one = np.eye(3);

# #compute dampings
# #keep it simple, stupid!
# def calc_damping(k,mass,overdamping=0.):
#     dampings = np.sqrt(np.divide(k, mass)) + overdamping
#     return dampings

# #just don't: after running 1 time step, construct a constant diagonal vertex damping matrix as a function of csr_K
# # dampings = np.divide(calc_damping(lam,mass,overdamping=0.3),mass)
# dampings = 200 + 0.*dampings #set it to 200mass #

# plt.hist(mass)
# plt.show()

In [ ]:
# # this takes 0.5 seconds to compute the net forces on 500 rows.  Not ideal
# # a number of functions need to be njit'd to get this down to ~30ms like pressure
# def calc_elastic_forces(net_elastic_forces, mesh, comp, outward_normals):
#     #for each face
#     for fid, face in enumerate(mesh.faces):
#         vid0,vid1,vid2 = mesh.faces[fid]
#         trim = np.array(mesh.triangles[fid])
#         tris = np.array(comp.triangles[fid])
#         #compute elastic nodal forces
#         N1N2N3 = outward_normals[fid]
#         nf0, nf1, nf2 = calc_elastic_force(trim, tris, calc_P, N1N2N3)
#         net_elastic_forces[vid0] += nf0
#         net_elastic_forces[vid1] += nf1
#         net_elastic_forces[vid2] += nf2

damping_const

# [SIMULATION] Use FEI time stepping to slowly integrate this ode over a long period of time
- Experimental design: isobaric expansion for small spring forces.


In [ ]:
#load desired mesh
os.chdir(f'{nb_dir}/Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_400.stl')
#subtract the center of mass
mesh.vertices -= mesh.center_mass

#set the initial volume
V_initial = 26 #cm^3
# mesh.vertices /= np.cbrt((mesh.volume)*3/(4*np.pi))
mesh.vertices *= np.cbrt(V_initial/mesh.volume)

#normalize the mean radius to R
# R = 1. #unit length
# mesh.vertices *= R/np.mean(np.linalg.norm(mesh.vertices,axis=1))

assert(mesh.is_watertight)
assert(mesh.is_winding_consistent)

# mean radius check
print (f"initial volume is {mesh.volume:.1f} cm^3.")
print (f"which has a mean radius of {np.mean(np.linalg.norm(mesh.vertices,axis=1)):.1f} cm.")


In [ ]:
#set material parameters
pressure_mmhg = 5. #mmHg filling pressure
pressure = 1333.*pressure_mmhg*1e-6 # 1 g cm / ms^2 = 10^-6 dyn/cm^2dyn/cm^2


delta = 0.01; #cm 
mu = .01;   # 1 g cm / ms^2 = 10^-6 dyn/cm^2
lam = 1.;      #1 g cm / ms^2
one = np.eye(3);
density = 1.05 #g/cm^3
calc_P = get_calc_P(mu, lam, one, delta)

#k = 1.   # youngs modulus   3000.*10**4        #gcm/s^2 / cm
#nu= 1.   #poisson's ratio
# extension =1.0  #unitless
     
#compute constant vector of vertex masses
mass = np.array([get_mass(vid, mesh, density = density*delta) for vid in range(mesh.vertices.shape[0])])

#set damping params
crit_damp = np.around(np.sqrt(lam/np.mean(mass)*delta))  # 1/second
damping_const = crit_damp # damping_const = float(3*10**3)  # 1/second
print (f"critical damping would at around {crit_damp} per millisecond.")
print (f"damping was set to {damping_const:.1f} per millisecond.")
dampings = damping_const + 0.*mass


In [ ]:
#make file for logging results
output_dir = nb_dir+'/Data/Log'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

output_fn = f'unitful-elastic-expansion_Pfill_{pressure_mmhg:.1f}.csv'
output_file = os.path.join(output_dir, output_fn)
# output_file = get_incremented_output_filename(output_dir, output_fn)
with open(output_file, 'w') as opened_file:
    opened_file.write('time,volume,area,avg_Fs,avg_Fp,my_Q\n')
print(output_file)

In [ ]:
#reinitialize time
tme = 0.

assert(mesh.is_watertight)
assert(mesh.is_winding_consistent)
print("check average lengths:")
print(np.mean(np.linalg.norm(mesh.vertices,axis=1)))
print(np.cbrt(mesh.volume*3/(4*np.pi)))

comp = mesh.copy()#comp contains tris, mesh contains trim
vertices = np.array(comp.vertices)
velocity = 0.*np.array(comp.vertices)
vertex_indices_of_caps = []

# # identify vertices that are on the caps (and therefore do not move)
# # caps = trimesh.load('Data/patient85_RA_caps.ply')
# mesh_closed = trimesh.load('Data/patient85_RA_closed.ply')
# mesh_closed.vertices /= 10 #convert length units from mm to cm
# df = pd.read_csv('Data/patient85_RA_vertices.csv',index_col = 0)
# vertex_indices_of_not_caps = list(df.query('group == -1').index)
# vertex_indices_of_caps = list(df.query('group > -1').index)
# mesh_open = mesh_closed.submesh((vertex_indices_of_not_caps,), append=True)
# mesh_caps = mesh_closed.submesh((vertex_indices_of_caps,), append=True)
# mesh = mesh_closed.copy()

#remove connections that are on caps
#note that this doesn't remove connections from non caps 
# to the caps, which is good.
VN = mesh.vertex_neighbors
for vid in vertex_indices_of_caps:
    VN[vid] = []
print(f"initial mesh volume is {mesh.volume} cm^2.")

In [ ]:
#precompute material space normals
outward_normals = 0.*mesh.triangles.copy()
for fid, trim in enumerate(mesh.triangles):
    N1N2N3 = calc_outward_normals(trim)
    outward_normals[fid] = N1N2N3
#assert one of the normals is in fact a unit vector
assert(np.isclose(np.linalg.norm(outward_normals[1][1]),1.))

In [ ]:
zero_array = 0.*np.array(mesh.vertices)
velocity = zero_array.copy().T

net_elastic_forces = zero_array.copy()
calc_elastic_forces(net_elastic_forces, mesh, comp, outward_normals)

In [ ]:
# ym2 is negative.

In [ ]:
h = 10**-3      #time step in seconds 
#advance in time and record data
recording = True
nsteps = int(1.*10**5)#2*10**5#6*10**1#7*10**3#2
printProgressBar(0, nsteps, prefix = 'Progress:', suffix = 'Complete', length = 50)
zero_txt = 0.*np.array(mesh.vertices)
velocity = zero_txt.copy()
dqdt_old = velocity.copy()
dvdt_old = velocity.copy()
N = mesh.vertices.shape[0] #number of vertices
zero_txt = np.zeros((N,N)).astype('float32')
zero_pos = np.array([0.,0.,0.]).astype('float32')


with open(output_file, 'a') as opened_file:
    for step in range(nsteps):
        vertices = np.array(comp.vertices)

        #compute pressure forces - takes 40ms - njit'd
        pressure_forces = pressure * np.array([get_A(vid, comp) for vid in range(N)])

        #compute elastic forces - takes 600ms - not njit'd
        net_elastic_forces = zero_array.copy()
        calc_elastic_forces(net_elastic_forces, mesh, comp, outward_normals)
        Fnet = net_elastic_forces+pressure_forces
        
        #explicitely fix any vertex positions (none for sphere)
        #for vid in vertex_indices_of_caps:
        #    Fnet[vid] = zero_pos
    
        #integrate in time
        dvdt       = np.divide(Fnet.T,mass) - np.multiply(dampings,velocity.T) #FEI
#         dqdt       = np.divide(2 * momentum.T + dpdt.T , mass).T  # IMR is self adjoint and second order accurate
        dqdt       = velocity + h * dvdt.T / 2 #IMR
#         dqdt = dpdt #viscous motion approximation with trivial masses
        
#         comp.vertices += h   * dqdt
#         comp.vertices += h/2 * (dqdt_old + dqdt)
        #         momentum      += h/2 * (dpdt_old + dpdt)
        comp.vertices += h * dqdt
        #comp.vertices += h/2 * (dqdt_old + dqdt)
        
        dvdt_old = dvdt.copy()
        dqdt_old = dqdt.copy()
        velocity  += h   * dvdt.T

        tme += h

        if recording:
            #measure desired observables
            volume = comp.volume
            area   = comp.area
            avg_Fs, avg_Fp = mean_magnitudes(net_elastic_forces,pressure_forces)
            my_Q   = get_Q(comp)
             
            #contiguous saving
            stdout = tme, volume, area, avg_Fs, avg_Fp, my_Q
            stdout = str((tme, volume, area, avg_Fs, avg_Fp, my_Q)).strip('()')+'\n'
            opened_file.write(stdout)
            
            #structured saving
            # outdict = {'time':tme,
            #     'volume':volume,
            #     'area':area,
            #     'avg_F_elastic':avg_Fs,
            #     'avg_F_pressure':avg_Fp,
            #     'my_Q':my_Q}

        printProgressBar(step + 1, nsteps, prefix = 'Progress:', suffix = 'Complete', length = 50)
beep(5)

volume = comp.volume
area   = comp.area
avg_Fs, avg_Fp = mean_magnitudes(net_elastic_forces,pressure_forces)
my_Q   = get_Q(comp)
print(f"curent time is {tme:.5f} milliseconds.")
print(f"the volume is {volume:.6f} cm^3.")
print(f"the avg spring force is {avg_Fs:.4f} gcm/ms^2.")
print(f"the avg pressure force is {avg_Fp:.4f} gcm/ms^2.")
print(f"the avg damping force is {np.mean(np.multiply(dampings,velocity.T).T):.4f} gcm/ms^2.")

In [ ]:
print((step,tme))
beep(3)

# quicklook at most recent state

In [ ]:
# unit length changing?
print('initially')
r_init1 = np.mean(np.linalg.norm(mesh.vertices,axis=1)); r_init2 = mesh.volume#np.cbrt(mesh.volume*3/(4*np.pi))
print(r_init1)
print(r_init2)

# unit length changing?
print('finally')
r_final1 = np.mean(np.linalg.norm(comp.vertices,axis=1)); r_final2 = comp.volume#np.cbrt(comp.volume*3/(4*np.pi))
print(r_final1)
print(r_final2)

# percent increase in R at end of simulation
print('percent increase')
print(100 * (r_final1-r_init1)/r_init1) 
print(100 * (r_final2-r_init2)/r_init2) 

In [ ]:
# comp.show()

# plot all these net forces

In [ ]:
x, y, z = mesh.vertices.T
u, v, w    = pressure_forces.T
us, vs, ws = net_elastic_forces.T
mean_magnitudes(net_elastic_forces,pressure_forces)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.gca(projection='3d')
ax.quiver(x, y, z, u, v, w, length=.002, normalize=False, color='blue', label='pressure forces')
ax.quiver(x, y, z, us, vs, ws, length=.002, normalize=False, color='orange', label='elastic forces')
ax.legend()
plt.show()


_Note_ First data run was not properly saved.  Rerun tonight with relevant parameters, to do that, I need to relate wall stress to mainly either the first or second lame parameter. my prediction, $\lambda$

# DONE: find out whetehr final deformation is dominated by the second lame parameter, $\lambda$
- _Answer_ they're about even.  set them equal to $\sim10^5$ dyn/mm$^2$.


In [ ]:
#first, test if there's a rough pressure-elastic force balance
fe,fp = mean_magnitudes(net_elastic_forces,pressure_forces)
print(f'Ratio of pressure forces to net nodal elastic forces is {fp/fe:.5f}')

In [ ]:
# compute the magnitude of either summand from the constitutive model
print (2*mu*np.linalg.det((S - one)))
print ( lam*np.trace(S - one) )

In [ ]:
lstmu = []
lstlam= []
for fid, tris in enumerate(comp.triangles):
    trim = mesh.triangles[fid]
    S, R = get_SR(trim,tris)
    lstmu.append(2*mu*np.linalg.norm((S - one)))
    lstlam.append(lam*np.trace(S - one) * np.linalg.norm(one))

In [ ]:
print (np.mean(np.array(lstmu)))
print (np.mean(np.array(lstlam)))

# plot time series in output_file
- #TODO(in a bit): add units relevant to cardiac mechanics 
- #TODO: run the sim again overnight

In [ ]:
#did results save to this file? Yes
output_file = '/Users/timothytyree/Documents/GitHub/care/notebooks/rapids-notebooks/Data/Log/unitful-elastic-expansion_Pfill_5.0.csv'
df = pd.read_csv(output_file)
df.dropna(inplace=True)

In [ ]:
df.head()
df['svr'] = df['area']/df['vovo']

In [ ]:
fontsize=16
yfield = 'volume'
x_values = df['time'].values
y_values = df[yfield].values
fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot()
ax.scatter(x=x_values,y=100*(y_values-y_values[0])/y_values[0])
ax.set_ylim([-0.001,0.0025])
ax.set_xlim([-0.001,.4025])
# ax.set_title('isobaric expansion in the overdamped limit',fontsize =fontsize)
ax.set_ylabel(f'percent change \nin {yfield} (\%)',fontsize =fontsize)
ax.set_xlabel('time (sec)',fontsize =fontsize)
fig.tight_layout()
# savefig_file = 'fig/plot_'+yfield+'_'+output_file.split('/')[-1].replace('.csv','.png')
# fig.savefig(savefig_file, dpi = 300)
#fig.show()

In [ ]:
fontsize=16
yfield = 'area'
x_values = df['time'].values
y_values = df[yfield].values
fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot()
ax.scatter(x=x_values,y=100*(y_values-y_values[0])/y_values[0])
ax.set_ylim([-0.001,0.25])
ax.set_xlim([-0.001,.4025])# ax.set_title('isobaric expansion in the overdamped limit',fontsize =fontsize)
ax.set_ylabel(f'percent change \nin {yfield} (\%)',fontsize =fontsize)
ax.set_xlabel('time (sec)',fontsize =fontsize)
#fig.show()
fig.tight_layout()
savefig_file = 'fig/plot_'+yfield+'_'+output_file.split('/')[-1].replace('.csv','.png')
# fig.savefig(savefig_file, dpi = 300)

In [ ]:
fontsize=16
yfield = 'avg force'
x_values = df['time'].values
fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot()
y_values = df['avg_Fs'].values
ax.scatter(x=x_values,y=y_values)
# ax.scatter(x=x_values,y=100*(y_values-y_values[0])/y_values[0])
y_values = df['avg_Fp'].values
ax.scatter(x=x_values,y=y_values)
# ax.scatter(x=x_values,y=100*(y_values-y_values[0])/y_values[0])
ax.set_ylim([-0.001,0.0025])
ax.set_xlim([-0.001,.4025])
# ax.set_title('isobaric expansion in the overdamped limit',fontsize =fontsize)
ax.set_ylabel(f'percent change \nin {yfield} (\%)',fontsize =fontsize)
ax.set_xlabel('time (sec)',fontsize =fontsize)
#fig.show()
fig.tight_layout()
savefig_file = 'fig/plot_'+yfield+'_'+output_file.split('/')[-1].replace('.csv','.png')
# fig.savefig(savefig_file, dpi = 300)

In [ ]:
1+1

In [ ]:
#